In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shapefile as shp 

In [3]:
map_results_folder = 'map_results' # CHANGE TO 'map_results_onshore' for onshore results

## Instructions for `get_elcc_map` function
----------------
Every elcc map can be found and opened directly in the `map_results` folder. Those in `map_results_onshore` have the offshore locations replaced with `nan` values.

Parameters are as follows:
- `region` (str): lower case region {'basin', 'california', 'mountains', 'northwest', 'southwest'}
- `year` (int): {2016, 2017, 2018, 2019}
- `capacity` (int): nameplate generator capacity in MW (our base case is 500 MW) {100, 500, 2000}
- `technology` (string): {'wind', 'solar'}
- `add_on` (string): LEAVE as None for base cases {'FF' (fixed FOR), 'storage' (250 MW storage)}

In [4]:
def get_elcc_map(region,year,capacity,technology,add_on=None):
    elcc_map_file = '%s/%s_%d_%d_MW_%s%s_results_map.csv' % (map_results_folder,region,year,capacity,technology,'' if add_on is None else '_'+add_on)
    elcc_df = pd.read_csv(elcc_map_file,index_col=0,header=0)
    elcc = elcc_df.to_numpy()
    lats = np.asarray(elcc_df.index.values)
    lons = np.asarray(elcc_df.columns.values).astype(float)
    return elcc, lats, lons 

In [66]:
def plot_map(ax, elcc, lats, lons):
    # kwargs:
    #   outline (bool)

    sns.set_theme(style="whitegrid")

    sf = shp.Reader("state_lines/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")

    outline_color='white'
    outline_width=1
    
    for shape in sf.shapeRecords():
        npoints=len(shape.shape.points) # total points
        nparts = len(shape.shape.parts) # total parts

        if nparts == 1:
            x = [i[0] for i in shape.shape.points[:]] #- np.min(lons)
            y = [i[1] for i in shape.shape.points[:]] #- np.min(lats)
            ax.plot(x,y,color=outline_color,linewidth=outline_width)
        else:
            for regionPart in range(0,nparts):
                if regionPart < (nparts-1):
                    endIndex = shape.shape.parts[regionPart+1]
                else:
                    endIndex = npoints
                x = [i[0] for i in shape.shape.points[shape.shape.parts[regionPart]:endIndex]] #- np.min(lons)
                y = [i[1] for i in shape.shape.points[shape.shape.parts[regionPart]:endIndex]] #- np.min(lats)
                ax.plot(x,y,color=outline_color,linewidth=outline_width)

    #contour plot
    im = ax.contourf(elcc,vmax=100,vmin=0,cmap='plasma', extent = [np.min(lons),np.max(lons),np.min(lats),np.max(lats)])
    ax.contour(elcc,vmax=100,vmin=0,colors='b', linewidths=.5, extent = [np.min(lons),np.max(lons),np.min(lats),np.max(lats)])
    

    ax.set_xlim(np.min(lons),np.max(lons))
    ax.set_ylim(np.min(lats),np.max(lats))
    
    # colorbar
    
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.outline.set_visible(False)

    cbar.ax.tick_params(labelsize=8)
    cbar.ax.tick_params(axis='y',length=2,width=1)

    # axis 
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)


    return ax

In [67]:

    fig, axs = plt.subplots(5,2,dpi=240, figsize=(7,10),constrained_layout=True)

    capacity = 500

    i = 0
    
    for region in ['basin','california','mountains','northwest','southwest']:
        for year in [2019]:
            for technology in ['solar','wind']:
                ax = axs[i//2,i%2]
                elcc, lats, lons = get_elcc_map(region,year,capacity,technology)

                ax = plot_map(ax,elcc,lats,lons)
                title = '%s %s' % (region.capitalize(),technology.capitalize())
                ax.set_title(title)

                i +=1
    

    img_name = '../images/figure_1_CV_maps'
    plt.savefig(img_name)  
    plt.close()           
